[TOC]

## RAG介绍
检索增强生成（Retrieval Augmented Generation），简称RAG，正逐渐成为大型语言模型（LLM）应用领域的热门解决方案。随着近期大型模型的兴起，人们开始认识到这些模型的强大潜力。然而，在将这些模型应用于具体的业务环境中时，我们发现通用的基础模型往往难以满足特定的业务需求。这主要是由于以下几个方面的挑战：

- **知识覆盖的限制**：大型模型的知识库主要基于其训练数据，而这些数据通常来源于互联网上的公开信息。对于需要实时更新、保密或特定场景下的数据，模型可能无法获取，从而限制了其在这些领域的应用能力。
- **生成内容的准确性问题**：由于大型模型的输出基于数学概率和数值计算，有时可能会产生不准确或不合逻辑的内容，尤其是在模型缺乏相关知识的领域。区分这些内容的准确性对用户提出了较高的要求。
- **数据安全与隐私保护**：企业对数据安全和隐私保护有着严格的要求。依赖于通用大模型的应用方案可能会面临数据泄露的风险，这迫使企业在使用这些模型时不得不在数据安全和效果之间做出权衡。

RAG技术通过结合检索和生成两个阶段，有效地解决了上述问题。它不仅能够利用现有的知识库提供准确的信息，还能够保护企业的数据安全，避免敏感信息的泄露。通过这种方式，RAG为大型模型在实际业务场景中的应用提供了一种新的、更为可靠的解决方案。

## RAG组成
<img src="./rag_flow.png" width="680px">
如上图，RAG（检索增强生成）技术通常可以分为两个主要阶段：检索阶段和生成阶段。下面详细介绍这两个阶段及其作用：

1. **检索阶段**：
   在检索阶段，RAG系统的目标是从大量的数据源中快速准确地找到与用户查询最相关的信息。这个阶段通常涉及以下几个步骤：
   - **数据准备**：首先，需要对数据源进行预处理，包括数据清洗、标注、向量化等，以便能够高效地进行检索。
   - **索引构建**：通过向量化或其他编码技术，将数据转换为机器可理解的格式，并构建索引，以便快速检索。
   - **相似度匹配**：当用户提出查询时，系统会将查询转换为向量表示，并在索引中寻找最相似的文档或数据片段。
   - **信息召回**：根据相似度匹配的结果，系统召回最相关的信息，这些信息将用于辅助生成阶段。

2. **生成阶段**：
   生成阶段利用检索阶段召回的信息来生成用户查询的响应。这个阶段的主要作用包括：
   - **上下文融合**：将召回的信息与用户的原始查询结合起来，形成一个丰富的上下文，为生成模型提供更多的背景知识。
   - **文本生成**：使用预训练的语言生成模型（如GPT、T5等），根据融合的上下文生成自然语言响应。生成模型会考虑上下文中的信息，以产生准确、相关且连贯的文本。
   - **质量优化**：对生成的文本进行后处理，包括去除重复内容、调整句子结构、压缩和摘要、风格一致性调整等，以提高文本的可读性和准确性。

在RAG系统中，生成阶段发挥着至关重要的作用，它不仅需要结合检索阶段的结果，还需要利用先进的自然语言处理技术来产生高质量的输出。通过这种方式，RAG技术能够提供更加准确、丰富且用户友好的文本回答，满足用户在各种复杂场景下的信息需求。

接下来，将主要介绍生成阶段设计的流程与技术，并用案例帮助大家理解。

## 生成阶段
如上图（RAG流程图）中，假设我们已从外部数据库中检索出和用户query相似的内容（relevant information），接下来可将用户query、外部相关信息注入提示模板Prompt中。然后将prompt输入大模型LLM得到问题回复。

在RAG（Retrieval-Augmented Generation）的应用中，Prompt作为输入给大型语言模型的信息，对模型生成的准确性起着至关重要的作用。在这种场景下，Prompt通常由几个部分组成：任务的描述、通过检索获得的背景知识，以及具体的用户问题作为任务指令。根据具体的任务需求和所使用的大型模型的特性，还可以在Prompt中添加额外的指令来进一步提升模型输出的质量。例如，在一个基础的知识问答任务中，一个典型的Prompt可能包括以下元素：

   - 任务描述：简要说明需要模型完成的任务，比如“请根据以下信息回答相关问题。”
   - 背景知识：提供检索到的相关数据或信息，用于丰富模型的上下文理解。
   - 任务指令：明确提出用户的问题。

通过这样的结构化Prompt，RAG系统能够有效地引导大型语言模型，结合检索到的信息，生成准确、详尽的回答。

**结构化Prompt举例：** （商品客服问答场景）

```markdown
[任务描述]

你是一个专业的客服机器人，请参考 [背景知识] 回答 [问题] 。

[背景知识]

{relevant information} // 检索阶段返回的相关信息

[问题]

{用户提问的 query} // eg. 有几种四件套？
```

有了上述Prompt后，便可将其输入大语言模型，得到生成结果。下面将代码案例说明如何使用。

---

**例子**

1）python实现，利用LangChain开发框架

```python
#!/usr/bin/env python3
import os
import openai
# gpt 网关调用
os.environ["OPENAI_API_KEY"] = "{您的 api key}"
os.environ["OPENAI_API_BASE"] = "{您的 url}"
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

context = "1. 全棉床上四件套：材质：采用全棉材质，柔软舒适，对皮肤无刺激。设计：床单和被套的设计简约大方，色彩柔和，适合各种家居风格。尺寸：通常尺寸为203*229cm，适合双人使用，提供充足的睡眠空间；2. 萌趣布朗熊图案床上四件套：图案：萌趣的布朗熊图案增添童趣，让卧室焕发活力。材质：全棉面料，透气性好，柔软舒适；3. 学生宿舍专用床上四件套：适用性：适合学生宿舍单人或双人使用。材质与设计：纯棉材质，简约设计，易于搭配，打造舒适睡眠环境；4. 双面印花床上四件套：设计：A面采用可爱小动物和冰淇淋图案装饰，B面灵动波点，彰显少女甜美。材质：全棉面料，舒适柔软；5. 澳毛大豆纤维填充被芯套：填充物：采用51%澳毛和20%大豆纤维填充，保暖性能优良。设计：子母被设计，方便拆洗和更换，适合不同季节使用。"

prompt = ChatPromptTemplate.from_template(
    '''
        【任务描述】
        请根据用户输入的上下文回答问题，并遵守回答要求。

        【背景知识】
        {context}

        【回答要求】
        - 你需要根据背景知识的内容回答。
        - 对于不知道的信息，直接回答“未找到相关答案”
        -----------
        {question}
    '''
)

model = ChatOpenAI(model="gpt-35-turbo-1106")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"context":context, "question": "有几种四件套？"})
```

2） 上述代码执行结果：

```markdown
'一共有五种四件套，分别是全棉床上四件套、萌趣布朗熊图案床上四件套、学生宿舍专用床上四件套、双面印花床上四件套和澳毛大豆纤维填充被芯套。'
```

## 后处理

后处理是RAG系统中的一个关键环节，通常发生在检索阶段之后，旨在优化和提升检索和生成结果的质量和相关性。
<img src="./postprocess.png" width="680px">
* Embedding Model : 将文档段落编码为向量的嵌入模型。
* Retriever：通过嵌入模型编码用户问题query，并返回嵌入问题附近的任意编码文档documents。
* 后处理（可选）: Compressor提取关键信息；ReRanker根据某规则重新计算query与documents间的得分。
* Language Model : 语言模型用于接收来自检索器或重排器的记录以及问题，并返回答案。
  

**为什么进行后处理？**
1. **优化信息密度**：由于LLM对输入的字数有限制，后处理可以通过筛选和压缩信息从大量检索结果中提炼出最核心的信息。这样可以确保即使在字数限制内，也能向模型提供最相关和最有价值的数据，从而生成高质量和高密度的信息内容。

2. **提高生成效率**：在有限的字数下，模型需要处理的信息越精炼，其生成答案的效率就越高。后处理通过去除冗余和不相关信息，确保模型专注于最关键的内容，这样可以在有限的交互中快速生成准确和有用的回答。

3. **确保内容的连贯性**：在字数受限的情况下，模型可能无法一次性接收并处理所有相关信息。后处理可以帮助组织和结构化信息，使其在生成阶段能够以连贯的方式呈现，避免因字数限制而导致的信息断层或不完整。

4. **提升生成文本的质量**：后处理不仅可以优化输入数据，还可以通过调整生成策略来提升输出文本的质量。例如，通过设置优先级，确保最重要的信息首先被包含在生成的文本中，即使在字数限制的情况下也能保证回答的核心价值。


后处理通常可包括以下方面：

1. **信息压缩**：由于检索阶段可能会返回大量相关信息，为了提高生成阶段的效率和性能，可对大量信息进行压缩，提取最关键的内容。这可能涉及到提取关键句子、短语或概念，以便在生成答案时能够集中于最相关的信息。

2. **重新排序**：根据与用户查询的相关性对检索结果进行重新排序。这通常基于文档与查询的匹配程度、文档的权威性、用户的历史偏好等因素。通过这种方式，最相关和最有用的信息会被放在最前面，以供生成模型优先考虑。

<!-- 3. **过滤**：识别重复的信息或与查询不相关的噪声数据，并去除这些内容，从而确保生成的文本是独特和有价值的。 -->

通过这些后检索处理步骤，RAG系统能够确保生成阶段的输入是精炼、相关且高质量的，从而提高最终生成文本的准确性和用户满意度。为了帮助理解上述后处理步骤，请看下面提供的案例。

### 案例

```markdown
[背景] ：用户对一款新型智能手机的功能和规格感兴趣，并提出了查询query。为了回答这个问题，RAG系统的检索阶段是从外部知识库中检索了相关的产品评测、技术规格表和用户评论。

[后处理阶段]
1. 信息压缩：由于检索到的文档数量庞大，首先对信息进行压缩即提取每篇文档中的关键句子和重要属性。如从技术规格表中提取了处理器型号、内存大小、屏幕分辨率等重要属性，且从用户评论中提取了关于电池续航、摄像头性能等的反馈。
2. 重新排序：根据用户查询的关键词和查询意图，对压缩后的信息进行了重新排序。如将用户评论中提到的电池续航问题放在前面，因为这是用户在购买决策中非常关心的一个方面。

[生成阶段]：系统将上述整合的上下文context输入到语言生成模型中，生成了一个涵盖了智能手机的主要功能、规格亮点以及使用反馈的回答。
```
<!-- 3. 过滤：识别并删除重复的信息，如多篇文档中都提到相同的功能点。且去除与查询不相关的内容（如与竞品的对比）。 -->

### 信息压缩
以往的方法在整合LLMs到检索式问答框架中存在一定的局限性，如计算成本高、对长文本的处理不足等。为了提高生成阶段的效率和性能，可对检索阶段返回的大量信息进行压缩，提取最关键的内容，以便在生成答案时能够集中于最相关的信息。


接下来将通过两个相关研究，探讨信息压缩的方法。

####  PRCA 方法
<img src="prca.png" width="600px">
PRCA（Pluggable Reward-Driven Contextual Adapter）提供了一种新颖的方法来提高黑盒大型语言模型在检索式问答任务中的性能，通过引入一个可训练的适配器和一个基于奖励的训练策略，使得模型能够更有效地利用检索到的信息来生成准确的答案。
PRCA可从检索器返回的Top-K相关文档中提炼简洁有效的上下文，作为生成器的输入。

- **适配器设计**：PRCA是一个可训练的适配器，它位于检索器和生成器之间（冻结检索器和生成器），通过token自回归策略来优化检索信息。
- **奖励驱动学习**：利用强化学习中的奖励信号来指导PRCA的训练，其中奖励是基于生成答案与真实答案之间的ROUGE-L分数。
- **两阶段训练策略**：
    <img src="prca_twoStage.png" width="400px">
    
  - **上下文提取阶段**：采用监督学习在文本摘要任务上进行预训练，学习从输入文本中提炼有效信息。
  - **奖励驱动阶段**：将生成器视为奖励模型，生成的答案与真实回复之间的差异可作为PRCA的奖励信号，使其更适合生成器生成准确答案。
  
  
#### RECOMP 方法

RECOMP（Retrieve, Compress, Prepend）作为RAG的一个中间步骤，它在上下文增强之前将检索到的文档压缩成文本摘要，然后再与上下文集成。这不仅降低了计算成本，还减轻了LMs在长文档中识别相关信息的负担。RECOMP介绍了两种压缩器——提取压缩器和抽象压缩器。这两种压缩器都经过训练，当生成的摘要被添加到 LM 的输入内容中时，可以提高 LM 在终端任务中的性能，同时保持摘要的简洁。如果检索到的文档与输入内容无关或没有为 LM 提供额外信息，我们的压缩器可以返回一个空字符串，从而实现选择性增强。

- **提取压缩器**：从检索到的文档中选择有用的句子。训练一个双编码器模型 $enc_{\theta}$，分别对输入上下文 $x_i$ 和候选句子 $S_i$ 进行编码。 通过对 [CLS] 标记的表示分别获得 $x_i$ 和 $S_i$ 的嵌入，并通过它们的内积计算相似度。对于每个输入查询 $x_i$，都要从检索到的文档中找出正向句子和反向句子，然后利用对比损失函数（contrastive loss）训练提取压缩器即通过最大化正向配对（$x_i$，$p_i$）之间的相似性，最小化负向配对（$x_i$，$N_i$）之间的相似性。
<img src="extractive.png" width="680px">
---

- **抽象压缩器**：通过综合多个文档的信息生成摘要。
    * 手动构建了四个提示prompts并利用教师模型GPT-3.5来生成文档集的摘要，并在其中选择每个示例（$s_t$）最终任务成绩最高的摘要作为目标摘要（第 4-8 行）。
    * $\text{Score}(M, y_i, [s_j; x_i])$ 与上述提取压缩器相同。然后比较在基础模型 M 上输入目标摘要和只输入 $x_i$（即不检索）的最终任务性能（第 6 行）。如果加入摘要使最终任务表现变差，则将目标摘要设置为空字符串（第 7 行），否则就会将目标摘要添加到训练集中（第 9 行）。这样就可以有选择性地增加内容，并降低预置无关文档的风险。
<img src="abstractive.png" width="680px">


**例子**：对prompt进行信息压缩

1）python实现，利用[LLMLingua](https://github.com/microsoft/LLMLingua)工具包

```python
question = "回答关于大模型的相关信息"
context =  [
    "Prompt【可选】◦告知LLM内system服从什么角色◦占位符：设置{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG（Retrieval-Augmented Generation），RAG 为了解决LLM中语料的通用和时间问题，通过增加最新的或者垂类场景下的外部语料，Embedding化后存入向量数据库，然后模型从外部语料中寻找相似语料辅助回复•Models◦可做 Embedding化，语句补全，对话等支持的模型选择，OpenAI为例•Parser【可选】◦StringParser，JsonParser 等◦将模型输出的AIMessage转化为string, json等易读格式上述介绍了Langchain开发中常见的components，接下来将通过一简单案例将上述组件串起来，让大家更熟悉Langchain中的组件及接口调用。",
    "LangChain 作为一个大语言模型（LLM）集成框架，旨在简化使用大语言模型的开发过程，包括如下组件： LangChain框架优点：1.多模型支持：LangChain 支持多种流行的预训练语言模型，如 OpenAI GPT-3、Hugging Face Transformers 等，为用户提供了广泛的选择。2.易于集成：LangChain 提供了简单直观的API，可以轻松集成到现有的项目和工作流中，无需深入了解底层模型细节。3.强大的工具和组件：LangChain 内置了多种工具和组件，如文档加载器、文本转换器、提示词模板等，帮助开发者处理复杂的语言任务。4.可扩展性：LangChain 允许开发者通过自定义工具和组件来扩展框架的功能，以适应特定的应用需求。5.性能优化：LangChain 考虑了性能优化，支持高效地处理大量数据和请求，适合构建高性能的语言处理应用。6.Python 和 Node.js 支持：开发者可以使用这两种流行的编程语言来构建和部署LangChain应用程序。由于支持 Node.js ，前端大佬们可使用Javascript语言编程从而快速利用大模型能力，无需了解底层大模型细节。同时也支持JAVA开发，后端大佬同样适用。本篇文章案例聚焦Python语言开发。",
    "LangChain表达式 (LCEL)LangChain表达式语言，或者LCEL，是一种声明式的方式，可以轻松地将链条组合在一起。 LCEL从第一天开始就被设计为支持将原型放入生产中，不需要改变任何代码，从最简单的“提示+LLM”链到最复杂的链(我们已经看到人们成功地在生产中运行了包含数百步的LCEL链)。以下是你可能想要使用LCEL的一些原因：流式支持 当你用LCEL构建你的链时，你可以得到最佳的首次到令牌的时间(输出的第一块内容出来之前的时间)。对于一些链，这意味着例如我们直接从LLM流式传输令牌到一个流式输出解析器，你可以以与LLM提供者输出原始令牌相同的速率得到解析后的、增量的输出块。异步支持 任何用LCEL构建的链都可以通过同步API(例如在你的Jupyter笔记本中进行原型设计时)以及异步API(例如在LangServe服务器中)进行调用。这使得可以使用相同的代码进行原型设计和生产，具有很好的性能，并且能够在同一台服务器中处理许多并发请求。优化的并行执行 无论何时，你的LCEL链有可以并行执行的步骤(例如，如果你从多个检索器中获取文档)，我们都会自动执行，无论是在同步接口还是异步接口中，以获得最小可能的延迟。重试和回退 为你的LCEL链的任何部分配置重试和回退。这是一种使你的链在大规模下更可靠的好方法。我们目前正在努力为重试/回退添加流式支持，这样你就可以在没有任何延迟成本的情况下获得增加的可靠性。访问中间结果 对于更复杂的链，通常在最终输出产生之前就能访问中间步骤的结果是非常有用的。这可以用来让最终用户知道正在发生什么，甚至只是用来调试你的链。你可以流式传输中间结果，它在每个LangServe服务器上都可用。输入和输出模式 输入和输出模式为每个LCEL链提供了从你的链的结构中推断出来的Pydantic和JSONSchema模式。这可以用于验证输入和输出，是LangServe的一个重要部分。无缝的LangSmith跟踪集成 随着你的链变得越来越复杂，理解在每一步究竟发生了什么变得越来越重要。 使用LCEL，所有步骤都会自动记录到LangSmith，以实现最大的可观察性和可调试性。"
]

from llmlingua import PromptCompressor

# llm_lingua = PromptCompressor()
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
    use_llmlingua2=True,
    device_map="cpu",
)
compressed_prompt = llm_lingua.compress_prompt(
    context,
    question=question,
    # Set the special parameter for LongLLMLingua
    condition_in_question="after_condition",
    reorder_context="sort",
    dynamic_context_compression_ratio=0.3, # or 0.4
    condition_compare=True,
    context_budget="+100",
    rank_method="longllmlingua",
)
compressed_prompt
    
```

2） 上述代码执行结果：

```markdown
{'compressed_prompt': 'Prompt【可选】◦告知LLM内system服从什么角色◦占位符{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG(Retrieval-Augmented,Embedding化后存入向量数据库 Embedding化,语句补全,OpenAI为例•Parser【可选】◦StringParser,JsonParser json等易读格式上述介绍了Langchain开发中常见的components\n\nLangChain 作为一个大语言模型)集成框架 LangChain框架优点:1.多模型支持 支持多种流行的预训练语言模型 OpenAI GPT-3、Hugging Face Transformers 提供了简单直观的API.强大的工具和组件,如文档加载器、文本转换器、提示词模板等.可扩展性.性能优化,适合构建高性能的语言处理应用。6.Python 和 Node.js 支持 Node.js,前端大佬们可使用Javascript语言编程从而快速利用大模型能力,后端大佬同样适用。本篇文章案例聚焦Python语言开发。',
 'compressed_prompt_list': ['Prompt【可选】◦告知LLM内system服从什么角色◦占位符{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG(Retrieval-Augmented,Embedding化后存入向量数据库 Embedding化,语句补全,OpenAI为例•Parser【可选】◦StringParser,JsonParser json等易读格式上述介绍了Langchain开发中常见的components',
  'LangChain 作为一个大语言模型)集成框架 LangChain框架优点:1.多模型支持 支持多种流行的预训练语言模型 OpenAI GPT-3、Hugging Face Transformers 提供了简单直观的API.强大的工具和组件,如文档加载器、文本转换器、提示词模板等.可扩展性.性能优化,适合构建高性能的语言处理应用。6.Python 和 Node.js 支持 Node.js,前端大佬们可使用Javascript语言编程从而快速利用大模型能力,后端大佬同样适用。本篇文章案例聚焦Python语言开发。'],
 'origin_tokens': 1510,
 'compressed_tokens': 324,
 'ratio': '4.7x',
 'rate': '21.5%',
 'saving': ', Saving $0.1 in GPT-4.'}
```

### 重新排序

1. *什么是重排？*

重新排序模型（也称为交叉编码器）是在给定查询和文档对的情况下，它将输出一个相似性得分。利用这个分数，按照与查询的相关性对文档重新排序。

长期以来，搜索工程师通常使用两阶段检索系统（检索器+重排序）。其中，第一阶段模型（嵌入模型/检索器）从一个更大的数据集中检索一组相关文档。然后，使用第二阶段模型（重排序器）对第一阶段模型检索到的文档进行重排序。

使用两个阶段是因为，从大型数据集中检索一小部分文档要比重新排序一大部分文档快得多--简单来说，重排序器速度慢，而检索器速度快。

<img src="reranker.png" width="680px">


2. *为什么要重排？*

检索器（双编码器）准确度较低的直观原因是，双编码器必须将文档的所有可能含义压缩到一个单一的向量中，这意味着会丢失信息。此外，双编码器没有查询的上下文，因为在用户查询之前就创建了嵌入（在收到查询之前并不知道查询内容）。

另一方面，重排器可以将原始信息直接接收到Transformer的计算中，这意味着信息损失更少。由于我们是在用户查询时运行检索器，因此还能根据用户查询的具体情况分析文档的含义，而不是试图生成一个通用的、平均的含义。

重排器避免了双编码器的信息损失，但也带来了不同的代价--时间。因此采用两阶段的检索（嵌入模型/检索器，重排器）做到时间和精度上的权衡。

3. *重排有什么选择？*

在检索增强生成（RAG）中，重排是一个关键步骤，它有助于优化检索到的文档的顺序，从而提高生成答案的准确性和相关性。重排的选择可以归纳为以下几种：

* 重新排序模型：这些模型通过考虑文档和查询之间的交互特征来评估它们的相关性。它们通常使用交叉熵损失进行优化，输出的是相关性得分而不是嵌入得分。例如，Cohere提供的在线模型和开源的bge-reranker-base与bge-reranker-large模型都属于这一类。

* 使用LLM作为重排器：随着大型语言模型（LLM）的出现，利用LLM进行重排成为可能。这种方法可以分为三类：对LLM进行微调以适应重新排序任务、提示LLM进行重排以及在训练过程中使用LLM进行数据增强。例如，RankGPT是一种利用LLM执行段落重排的方法，它采用排列生成方法和滑动窗口策略来有效地对段落进行重新排序。

* 基于内容的重排：这种方法考虑文档内容的质量，将重要的内容放在上下文的前面和后面，以提高模型对重要信息的关注度。

* 基于关联性的重排：这种方法考虑上下文内容之间的关联性，通过聚类和消重来提高信息的连贯性和多样性，减少信息的不一致性和冲突。

* 基于评价指标的重排：在选择最佳嵌入和重排模型时，会使用特定的评价指标，如命中率和平均倒数排名（MRR），来确定最佳的重排策略。

这些方法可以单独使用，也可以结合起来，以实现最佳的重排效果。开发者可能需要根据具体的应用场景和需求，尝试不同的重排策略和模型组合，以达到最优的性能。


4. *案例*

接下来，将通过引出MMR ReRanker的代码例子帮助大家理解ReRanker的作用。

MMR（Maximal Marginal Relevance）是一种在推荐系统中常用的多样性重排策略，旨在优化推荐列表的多样性，同时保持推荐内容的相关性。该策略通过在推荐列表中引入多样性约束，避免推荐结果过于集中或雷同，从而提高用户的满意度和系统的公平性。

MMR策略的优势在于它能够在保持推荐内容相关性的同时显著提升推荐列表的多样性。这种方法特别适用于处理那些需要平衡多种因素的推荐场景，如搜索引擎结果的重排、信息流的优化等。

**例子**：对照普通Retriever返回的TOP3文档和加入MMR ReRanker的Retriever返回的TOP3文档

1）python实现，利用LlamaIndex工具包

```python
import os

# gpt 网关调用
os.environ["OPENAI_API_KEY"] = "{您的 api key}"
os.environ["OPENAI_API_BASE"] = "{您的 url}"

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-35-turbo-1106", temperature=0.2)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# load data and index
documents = SimpleDirectoryReader("./data/paul_graham1/").load_data()
index = VectorStoreIndex.from_documents(documents)

# common retriever
retriever = index.as_retriever(
    similarity_top_k=3
)
nodes = retriever.retrieve(
    "作者在Y Combinator工作期间都做了些什么?"
)

from llama_index.core.response.notebook_utils import display_source_node

for n in nodes:
    display_source_node(n, source_length=200)

    
print('*'*18)

# with mmr reranker

mmr_retriever = index.as_retriever(
    vector_store_query_mode="mmr",
    similarity_top_k=3,
    vector_store_kwargs={"mmr_threshold": 0.5},
)
mmr_nodes = mmr_retriever.retrieve(
    "作者在Y Combinator工作期间都做了些什么?"
)

for n in mmr_nodes:
    display_source_node(n, source_length=200)
    
```

2） 上述代码执行结果：

```markdown
Node ID: 4f5e1079-dfec-4991-bb74-04db2bfb01a5
Similarity: 0.8639099140052439
Text: 所以我们将名称改为Hacker News，并将主题改为任何能够激发智力好奇心的内容。

HN无疑对YC有好处，但它也是我工作中最大的压力来源。如果我所要做的就是选择和帮助创始人，生活将会很容易。这意味着HN是一个错误。当然，一个人工作中最大的压力来源至少应该是工作核心的接近之处。而我就像一个在马拉松比赛中痛苦的人，不是因为跑步的努力，而是因为我穿着不合脚的鞋子起了水泡。当我在YC处理一些紧急...

Node ID: e98f8028-7aa6-41b5-9626-deb875996e8e
Similarity: 0.8613203711612752
Text: 与此同时，我一直在和Robert和Trevor策划我们可以一起工作的项目。我错过了和他们一起工作的日子，似乎肯定有我们可以合作的事情。

当我们在3月11日的晚餐后走在Garden和Walker街的拐角处时，这三个线索汇聚在一起。去他的那些VC，他们花了这么长时间才做出决定。我们将成立自己的投资公司，真正实现我们一直在讨论的想法。我会资助它，Jessica可以辞职并为其工作，我们也会得到Ro...

Node ID: 6de135be-c425-4507-84c2-96b9e5706e10
Similarity: 0.8550478002286129
Text: [14]

YC最独特的一点是批量模型：一次资助一群初创公司，每年两次，然后花三个月的时间集中精力帮助它们。这部分我们是偶然发现的，不仅仅是隐含地，而是由于我们对投资的无知而明确地发现的。我们需要作为投资者的经验。有什么比一次资助一大堆初创公司更好的方法呢？我们知道本科生在夏天会在科技公司找到临时工作。为什么不组织一个夏季项目，让他们创办初创公司呢？我们不会为成为某种假冒投资者而感到内疚，因...

******************
Node ID: 4f5e1079-dfec-4991-bb74-04db2bfb01a5
Similarity: 0.43195495700262193
Text: 所以我们将名称改为Hacker News，并将主题改为任何能够激发智力好奇心的内容。

HN无疑对YC有好处，但它也是我工作中最大的压力来源。如果我所要做的就是选择和帮助创始人，生活将会很容易。这意味着HN是一个错误。当然，一个人工作中最大的压力来源至少应该是工作核心的接近之处。而我就像一个在马拉松比赛中痛苦的人，不是因为跑步的努力，而是因为我穿着不合脚的鞋子起了水泡。当我在YC处理一些紧急...

Node ID: 6de135be-c425-4507-84c2-96b9e5706e10
Similarity: -0.017994423769901957
Text: [14]

YC最独特的一点是批量模型：一次资助一群初创公司，每年两次，然后花三个月的时间集中精力帮助它们。这部分我们是偶然发现的，不仅仅是隐含地，而是由于我们对投资的无知而明确地发现的。我们需要作为投资者的经验。有什么比一次资助一大堆初创公司更好的方法呢？我们知道本科生在夏天会在科技公司找到临时工作。为什么不组织一个夏季项目，让他们创办初创公司呢？我们不会为成为某种假冒投资者而感到内疚，因...

Node ID: 4786fc5f-f507-42ba-bcd2-6612d11d2cca
Similarity: -0.0203627485256761
Text: 我只能记得他之前这样做过一次。在Viaweb的一天，当我因为肾结石痛得弯下腰时，他建议他应该带我去医院。这就是让Rtm给出主动建议所需要的。所以我非常清楚地记得他的确切话语。“你知道吗，”他说，“你应该确保Y Combinator不是你做的最后一件酷事。”

当时我不明白他的意思，但渐渐地我明白了，他是在说我应该辞职。这似乎是个奇怪的建议，因为YC做得很好。但如果有一件事比Rtm给出建议更罕...
```

## RAG&Fine-tuning

1. *RAG vs. Fine-tuning*

<img src="./rag&ft.png" width="300px">

* 动态数据
    * RAG : 直接更新检索知识库，无需频繁重新训练，适合动态变化的数据场景。
    * Fine-tuning : 存储静态数据，需要重新训练用于知识更新。
    
* 可解释性
    * RAG : 回复能追溯到具体数据来源，提供更高的可解释性和可追踪性。
    * Fine-tuning : 黑盒模型，不总能清楚模型为何做出此反应。

* 降低幻觉
    * RAG : 生成的回复基于检索到的实际内容，不易产生虚构。
    * Fine-tuning : 根据特定领域数据训练有助于减少幻觉，但在未训练过的输入上仍可能出现幻觉。
    
* 模型定制
    * RAG : 侧重于信息检索和融合外部知识，但无法充分定制模型行为。
    * Fine-tuning : 允许根据特定数据调整LLM行为、写作风格和领域知识。
    
2. *使用场景举例*

<img src="raft.png" width="880px">
其中 open-book代表RAG，closed book代表Fine-tuning, RAFT代表RAG+Fine-tuning。

介绍了上述RAG和Fine-tuning的区别，下面根据实际场景介绍上述方法的使用。Remark: RAG和Fine-tuning非互斥，可搭配使用。

* 总结(Summarization)
    * 需要特定的领域和写作风格，选择Fine-tuning

* 问题解答(Question answering) 
    * 针对公司内部文件的问题解答系统
    * 需要动态更新数据，选择RAG

* 客户支持聊天机器人(Customer support chatbot)
    * 回答电子商务网站的问题
    * 需要动态更新数据
    * 训练特定语调和行为
    * RAG + Fine-tuning

* 代码生成(Code Generation)
    * 基于私有和公共代码库的代码建议系统
    * 需要按照代码规范生成
    * RAG + Fine-tuning
    
大家可以根据具体的使用场景，选择上述两种方法或者组合使用。

## 参考引用

**返回参考引用的好处？**（Remark:需外部知识库的知识可公开、非隐私）

提高答案的准确性和可信度，促进知识的学习和传播，以及增强用户对模型生成内容的信任。

这有助于用户验证信息，培养批判性思维，并支持学术研究和内容创作。

下面将通过两个案例帮助大家理解 _参考引用_ 及其实现，分别是面对一个文档和多个文档，如何分chunks然后索引。

---

**例子一**：RAG的外部知识库包含单个文档，分割chunk，从chunks中寻找和用户提问相近的chunks，并输出引用内容。

1）python实现，利用LlamaIndex工具包

```python
#!/usr/bin/env python3

f = open('./data/paul_graham1/paul_graham_essay_chinese.txt', 'r')
lines = f.readlines()

from llama_index.core import Document
doc = Document(text=''.join(lines))

import os
# gpt 网关调用
os.environ["OPENAI_API_KEY"] = "{您的 api key}"
os.environ["OPENAI_API_BASE"] = "{您的 url}"

import openai
openai.api_key = os.environ['OPENAI_API_KEY']

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)
# per-index
index = VectorStoreIndex.from_documents(
    [doc], transformations=[text_splitter]
)

from langchain.llms import OpenAI
# 初始化LLM
llm = OpenAI(model="gpt-3.5-turbo")
# 用户的问题
user_question = "Sam Altman在这篇文章中做了什么？"
# 查询索引以获取相关文档
retriever = index.as_retriever(similarity_top_k=3)
response = retriever.retrieve(user_question)

def show_nodes(nodes, out_len: int = 200):
    for idx, n in enumerate(nodes):
        print(f"\n\n >>>>>>>>>>>> ID {n.id_}")
        print(n.get_content()[:out_len])
        
show_nodes(response)

```

2） 上述代码执行结果：

```markdown
 >>>>>>>>>>>> ID 7aea5a5d-d2ab-4bf9-a3cd-5e7c2f5dccf0
我过去常常飞往俄勒冈州看望她，我在那些飞行中有很多思考的时间。在其中一次飞行中，我意识到我准备好将YC交给别人了。

我问Jessica是否想成为总裁，但她不想，所以我们决定尝试招募Sam Altman。我们与Robert和Trevor进行了交谈，我们同意进行全面的管理层更迭。到目前为止，YC一直由我们四个人开始的原始LLC控制。但我们希望YC能够持续很长时间，要做到这一点，它不能由创始人控制。所


 >>>>>>>>>>>> ID 80e88981-bfb7-4fed-9ca0-e675e49f12b8
在印刷时代，发表文章的渠道非常小。除了一些被官方指定的思想家，他们参加纽约正确的派对，唯一被允许发表文章的人是专家撰写他们的专业。有许多文章从未被写过，因为没有办法发表它们。现在它们可以了，我将写它们。[12]

我做过几件不同的事情，但在我弄清楚该做什么工作的转折点上，是我开始在线发表文章的时候。从那时起，我知道无论我做什么，我都会一直写文章。

我知道在线文章最初会是一个边缘媒介。社会上它们看


 >>>>>>>>>>>> ID 922f2ccf-cb95-48df-a3e8-b289515e71fa
我当然有。所以在夏天结束时，Dan和我转而研究这种新的Lisp方言，我称之为Arc，在我在剑桥买的房子里。

接下来的春天，闪电击中了。我应邀在一次Lisp会议上发表演讲，所以我做了一个关于我们在Viaweb如何使用Lisp的演讲。之后，我将这个演讲的postscript文件放在了我的网站上，paulgraham.com，这是我多年前使用Viaweb创建的，但从未用于任何东西。在一天之内，它获得了
```

---

**例子二**：RAG的外部知识库包含多个文档，分割chunk，从chunks中寻找和用户提问相近的chunks，并输出chunk对应的文档链接及引用内容。


1）python实现，利用LlamaIndex工具包

```python
import os
# gpt 网关调用
os.environ["OPENAI_API_KEY"] = "{您的 api key}"
os.environ["OPENAI_API_BASE"] = "{您的 url}"
import openai
openai.api_key = os.environ['OPENAI_API_KEY']


from llama_index.readers.file import UnstructuredReader
from pathlib import Path
from llama_index.llms.openai import OpenAI
from llama_index.core import Document

reader = UnstructuredReader()
    
all_txt_files = [
    "data/background/component.txt",
    "data/background/intro.txt",
    "data/background/lcel.txt"
]

txt_contents = [
    "Prompt【可选】◦告知LLM内system服从什么角色◦占位符：设置{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG（Retrieval-Augmented Generation），RAG 为了解决LLM中语料的通用和时间问题，通过增加最新的或者垂类场景下的外部语料，Embedding化后存入向量数据库，然后模型从外部语料中寻找相似语料辅助回复•Models◦可做 Embedding化，语句补全，对话等支持的模型选择，OpenAI为例•Parser【可选】◦StringParser，JsonParser 等◦将模型输出的AIMessage转化为string, json等易读格式上述介绍了Langchain开发中常见的components，接下来将通过一简单案例将上述组件串起来，让大家更熟悉Langchain中的组件及接口调用。",
    "LangChain 作为一个大语言模型（LLM）集成框架，旨在简化使用大语言模型的开发过程，包括如下组件： LangChain框架优点：1.多模型支持：LangChain 支持多种流行的预训练语言模型，如 OpenAI GPT-3、Hugging Face Transformers 等，为用户提供了广泛的选择。2.易于集成：LangChain 提供了简单直观的API，可以轻松集成到现有的项目和工作流中，无需深入了解底层模型细节。3.强大的工具和组件：LangChain 内置了多种工具和组件，如文档加载器、文本转换器、提示词模板等，帮助开发者处理复杂的语言任务。4.可扩展性：LangChain 允许开发者通过自定义工具和组件来扩展框架的功能，以适应特定的应用需求。5.性能优化：LangChain 考虑了性能优化，支持高效地处理大量数据和请求，适合构建高性能的语言处理应用。6.Python 和 Node.js 支持：开发者可以使用这两种流行的编程语言来构建和部署LangChain应用程序。由于支持 Node.js ，前端大佬们可使用Javascript语言编程从而快速利用大模型能力，无需了解底层大模型细节。同时也支持JAVA开发，后端大佬同样适用。本篇文章案例聚焦Python语言开发。",
    "LangChain表达式 (LCEL)LangChain表达式语言，或者LCEL，是一种声明式的方式，可以轻松地将链条组合在一起。 LCEL从第一天开始就被设计为支持将原型放入生产中，不需要改变任何代码，从最简单的“提示+LLM”链到最复杂的链(我们已经看到人们成功地在生产中运行了包含数百步的LCEL链)。以下是你可能想要使用LCEL的一些原因：流式支持 当你用LCEL构建你的链时，你可以得到最佳的首次到令牌的时间(输出的第一块内容出来之前的时间)。对于一些链，这意味着例如我们直接从LLM流式传输令牌到一个流式输出解析器，你可以以与LLM提供者输出原始令牌相同的速率得到解析后的、增量的输出块。异步支持 任何用LCEL构建的链都可以通过同步API(例如在你的Jupyter笔记本中进行原型设计时)以及异步API(例如在LangServe服务器中)进行调用。这使得可以使用相同的代码进行原型设计和生产，具有很好的性能，并且能够在同一台服务器中处理许多并发请求。优化的并行执行 无论何时，你的LCEL链有可以并行执行的步骤(例如，如果你从多个检索器中获取文档)，我们都会自动执行，无论是在同步接口还是异步接口中，以获得最小可能的延迟。重试和回退 为你的LCEL链的任何部分配置重试和回退。这是一种使你的链在大规模下更可靠的好方法。我们目前正在努力为重试/回退添加流式支持，这样你就可以在没有任何延迟成本的情况下获得增加的可靠性。访问中间结果 对于更复杂的链，通常在最终输出产生之前就能访问中间步骤的结果是非常有用的。这可以用来让最终用户知道正在发生什么，甚至只是用来调试你的链。你可以流式传输中间结果，它在每个LangServe服务器上都可用。输入和输出模式 输入和输出模式为每个LCEL链提供了从你的链的结构中推断出来的Pydantic和JSONSchema模式。这可以用于验证输入和输出，是LangServe的一个重要部分。无缝的LangSmith跟踪集成 随着你的链变得越来越复杂，理解在每一步究竟发生了什么变得越来越重要。 使用LCEL，所有步骤都会自动记录到LangSmith，以实现最大的可观察性和可调试性。无缝的LangServe部署集成 任何用LCEL创建的链都可以使用LangServe轻松部署。"
]

doc_limit = 10

docs = []
for idx, f in enumerate(all_txt_files):
    if idx > doc_limit:
        break
    print(f"Idx {idx}/{len(all_txt_files)}")
    loaded_docs = reader.load_data(file=f, split_documents=True)
    loaded_doc = Document(
        id_=str(f),
        text=txt_contents[idx],
        metadata={"path": str(f)},
    )
    print(str(f))
    docs.append(loaded_doc)

# pip install unstructured

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

embed_model = OpenAIEmbedding(
    model_name="text-embedding-ada-002", api_key=os.environ["OPENAI_API_KEY"]
)

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")

from llama_index.core.storage.docstore import SimpleDocumentStore

for doc in docs:
    embedding = embed_model.get_text_embedding(doc.get_content())
    doc.embedding = embedding

docstore = SimpleDocumentStore()
docstore.add_documents(docs)


from llama_index.core.schema import IndexNode
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SummaryIndex
from llama_index.core.retrievers import RecursiveRetriever
import os
from tqdm.notebook import tqdm
import pickle


def build_index(docs, out_path: str = "storage/chunk_index"):
    nodes = []

    splitter = SentenceSplitter(chunk_size=512, chunk_overlap=70)
    for idx, doc in enumerate(tqdm(docs)):
        # print('Splitting: ' + str(idx))

        cur_nodes = splitter.get_nodes_from_documents([doc])
        for cur_node in cur_nodes:
            # ID will be base + parent
            file_path = doc.metadata["path"]
            new_node = IndexNode(
                text=cur_node.text or "None",
                index_id=str(file_path),
                metadata=doc.metadata
                # obj=doc
            )
            nodes.append(new_node)
    print("num nodes: " + str(len(nodes)))

    # save index to disk
    if not os.path.exists(out_path):
        index = VectorStoreIndex(nodes, embed_model=embed_model)
        index.set_index_id("simple_index")
        index.storage_context.persist(f"./{out_path}")
    else:
        # rebuild storage context
        storage_context = StorageContext.from_defaults(
            persist_dir=f"./{out_path}"
        )
        # load index
        index = load_index_from_storage(
            storage_context, index_id="simple_index", embed_model=embed_model
        )

    return index

index = build_index(docs)

out_top_k = 3

base_retriever = index.as_retriever(similarity_top_k=out_top_k)


def show_nodes(nodes, out_len: int = 512):
    for idx, n in enumerate(nodes):
        print(f"\n\n >>>>>>>>>>>> ID {n.id_}: {n.metadata['path']}")
        print(n.get_content()[:out_len])
        
query_str = "关于大模型的信息"

base_nodes = base_retriever.retrieve(query_str)

show_nodes(base_nodes)

```

2） 上述代码执行结果：

```markdown
>>>>>>>>>>>> ID b769edca-38c2-4abf-be03-6f6ea7947b58: data/background/intro.txt
LangChain 作为一个大语言模型（LLM）集成框架，旨在简化使用大语言模型的开发过程，包括如下组件： LangChain框架优点：1.多模型支持：LangChain 支持多种流行的预训练语言模型，如 OpenAI GPT-3、Hugging Face Transformers 等，为用户提供了广泛的选择。2.易于集成：LangChain 提供了简单直观的API，可以轻松集成到现有的项目和工作流中，无需深入了解底层模型细节。3.强大的工具和组件：LangChain 内置了多种工具和组件，如文档加载器、文本转换器、提示词模板等，帮助开发者处理复杂的语言任务。4.可扩展性：LangChain 允许开发者通过自定义工具和组件来扩展框架的功能，以适应特定的应用需求。5.性能优化：LangChain 考虑了性能优化，支持高效地处理大量数据和请求，适合构建高性能的语言处理应用。6.Python 和 Node.js 支持：开发者可以使用这两种流行的编程语言来构建和部署LangChain应用程序。由于支持 Node.js ，前端大佬们可使用Javascript语言编程从而快速利用大模型能力，无需了解底层大模型细节。同时也


 >>>>>>>>>>>> ID f6256766-4102-43f4-ab85-d77bd0b39902: data/background/component.txt
Prompt【可选】◦告知LLM内system服从什么角色◦占位符：设置{input}以便动态填补后续用户输入•Retriever【可选】◦LangChain一大常见应用场景就是RAG（Retrieval-Augmented Generation），RAG 为了解决LLM中语料的通用和时间问题，通过增加最新的或者垂类场景下的外部语料，Embedding化后存入向量数据库，然后模型从外部语料中寻找相似语料辅助回复•Models◦可做 Embedding化，语句补全，对话等支持的模型选择，OpenAI为例•Parser【可选】◦StringParser，JsonParser 等◦将模型输出的AIMessage转化为string, json等易读格式上述介绍了Langchain开发中常见的components，接下来将通过一简单案例将上述组件串起来，让大家更熟悉Langchain中的组件及接口调用。


 >>>>>>>>>>>> ID 71b28aba-a232-426b-a868-3b46ca2cbfbe: data/background/lcel.txt
重试和回退 为你的LCEL链的任何部分配置重试和回退。这是一种使你的链在大规模下更可靠的好方法。我们目前正在努力为重试/回退添加流式支持，这样你就可以在没有任何延迟成本的情况下获得增加的可靠性。访问中间结果 对于更复杂的链，通常在最终输出产生之前就能访问中间步骤的结果是非常有用的。这可以用来让最终用户知道正在发生什么，甚至只是用来调试你的链。你可以流式传输中间结果，它在每个LangServe服务器上都可用。输入和输出模式 输入和输出模式为每个LCEL链提供了从你的链的结构中推断出来的Pydantic和JSONSchema模式。这可以用于验证输入和输出，是LangServe的一个重要部分。无缝的LangSmith跟踪集成 随着你的链变得越来越复杂，理解在每一步究竟发生了什么变得越来越重要。 使用LCEL，所有步骤都会自动记录到LangSmith，以实现最大的可观察性和可调试性。无缝的LangServe部署集成 任何用LCEL创建的链都可以使用LangServe轻松部署。
```